Samuel Karkache
Trey Marcantonio

Differential Power Analysis metrics for Security Analysis of Integrated Circuits MQP. These DPA metric functions will be able to read multiple file types (ChipWhisper, LeCroyDSO, etc...) and perform different metric analysis on the associated power traces. This provides an interface for traces stored in different file formats and allows for efficient processing of large datasets. 

In [3]:
import numpy as np
from scipy import stats
import math
import matplotlib.pyplot as plt

In [ ]:
# Signal to Noise Ratio metric 
#   - labels: An array of arrays. Each index of the labels array (i.e. labels[0])
#     is a label group containing the corresponding power traces.
# return: the SNR signal 
def signal_to_noise_ratio(labels):
   # statistical mean and variances of each set 
   set_means = []
   set_variances = []
   
   for label in labels:
       set_means.append(np.mean(label, axis=0)) # take the mean along the column
       set_variances.append(np.var(label, axis=0)) # take the variance along the column
   
   # calculate overall mean and variance
   overall_mean= np.mean(set_means, axis=0)              
   overall_variance = np.var(set_variances, axis=0)

   # perform SNR calculation
   l_d = np.zeros(len(set_means[0]))
   for mean in set_means:
       l_d = np.add(l_d, np.square(np.subtract(mean, overall_mean)))
   l_n = overall_variance

   snr = np.divide(l_d, l_n)

   return snr

In [37]:
# Score metric: Ranks each key guess in a key partition based on a scoring function
#   - traces: The trace set to be evaluated
#   - score_fcn: Function callback that takes two arguments, traces and a guess candidate
#                and returns a "score" such that the higher the value, the more likely the 
#                key candidate is the actual key
#   - partitions: The number of partitions of the key full key.
# return: A 2D array rank. The value rank[i] are the key guess rankings for partition i. 
#         The value of rank[i][0] is the highest ranked key guess for partition i.
def score_and_rank(traces, score_fcn, key_candidates, partitions):
        ranks = []
        # for each key partition        
        for i in range(partitions): 
            dtype = [('key', int), ('score', 'float64')]
            partition_scores = np.array([], dtype=dtype)
            
            # for each key guess in the partition score the value and add to list
            for k in key_candidates:
                score_k = score_fcn(traces, k)
                key_score = np.array([(k, score_k)], dtype=dtype)
                partition_scores = np.append(partition_scores, key_score)
                
            # rank each key where partition_ranks[0] is the key that scored the highest
            partition_ranks = np.array([key_score[0] for key_score in np.sort(partition_scores, order='score')[::-1]])
            
            ranks.append(partition_ranks)
        return ranks

In [4]:
# Success Rate and Guessing Entropy Metric: Analyzes the security of a device by determining if the correct key was ranked number 1 for a given experiement
#   - correct_keys: An array of correct cryptographic keys. The value of correct_key[0] is the correct key for experiment 0
#   - ranks: The ranks of key guesses for a given experiment 
#   - num_experiments: The number of experiments conducted
# return: The values of success_rate and guessing_entropy for the given number of experiments
def success_rate_guessing_entropy(correct_keys, ranks, num_experiments):
    success_rate = 0
    guessing_entropy = 0
    
    # for each experiment
    for i in range(num_experiments):
        if ranks[i][0]  == correct_keys[i]:
            success_rate += 1 # add 1 to success rate if the correct key is ranked as number 1
        
        # guessing entropy is the log2 of the rank of the correct key
        guessing_entropy += math.log2(ranks[i].index(correct_keys[i]))
    
    success_rate = success_rate / num_experiments
    guessing_entropy = guessing_entropy / num_experiments
    
    return success_rate, guessing_entropy
    

In [ ]:
# T-test with TVlA metric:  In general |t| > th where th = 4.5 means that the system leaks information about the cryptographic key.
#   - fixed:  Trace set recorded with a fixed plaintext 
#   - random: Trace set recorded with a random set of plaintexts
# return: t_statistic and p-value
def t_test_tvla(fixed, random):
    
    # determine t_statistic and p-value using scipy 
    t_statistic, p_value = stats.ttest_ind(fixed, random)
    
    # high t-statistic and low p-values indicate that a given time sample leaks information
    return t_statistic, p_value